# CI7520 –Assignment 1: Classical Machine Learning - Part II - Clustering

**Submitted by:**

1) Adrian Bandy (K2132274)

2) Shashwat Bhardwaj (K2149137)

3) Royce Daran Shakespeare (K2046699)

4) Padmesh Upadhyay (K2136572)




Introduction

In this assignment we will be exploring a variety of clustering and classification models using sklearn and the breast cancer dataset.

In [150]:
#Importing The Required Libraries

#Data and data manipulation libraries
from sklearn import datasets
import pandas as pd
import numpy as np

#Visualisation
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.tree import plot_tree
from sklearn.inspection import plot_partial_dependence


#Data preprocessing
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler

#Models
from sklearn import cluster
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

#Hyperparameter Tuning and processing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_predict

#Scoring and Performance Evaluation
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import homogeneity_completeness_v_measure
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import (precision_recall_curve, PrecisionRecallDisplay)
from sklearn.metrics import homogeneity_score
from sklearn.metrics import completeness_score
from sklearn.metrics import v_measure_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

## Declared Functions

Using the DRY (Dont Repeat Yourself) method as outlined in The Pragmatic Programmer [1] we will declare functions that we use multiple times.
This has the benefit of less and cleaner code, which is far easier to update or maintain.

##### [1] Hunt, A., Thomas, D. and Cunningham, W., 2015. The pragmatic programmer. Boston: Addison-Wesley, Chapter 2.

In [151]:
def clustering_performance_evaluation(X, predicted_labels, print_output = False):
  '''
  Accepts data of shape 2d and the predicted labels
  Returns a series of different clustering scores
  If print_output = True it will print the scores as well as returning them
  '''
  #silhouette is mean intra-cluster distance - nearest-cluster distance
  #divided by the maximum of these two.
  #Values between 1 and -1. 1 being best score achievable.
  sil_score = silhouette_score(X, predicted_labels)
        
  #Calinski-Harabasz score
  #Ratio of sum of the inter-cluster dispersion and intra-cluster dispersion
  #Higher values better
  cal_har_score = calinski_harabasz_score(X, predicted_labels)
    
  #David-Bouldin Score
  #Avg similarity within cluster. 
  #Similarity is ratio of intra-cluster to inter-cluster
  #Low values are better (minima is 0)
  db_score = davies_bouldin_score(X, predicted_labels)
      
  if print_output == True:
      print(f"Silhouette Score: {sil_score:.3f}\
      \nCalinski-Harabasz Score: {cal_har_score:.3f}\
      \nDavid-Bouldin Score: {db_score:.3f}")
    
  if print_output == False:
      return [sil_score, cal_har_score, db_score]

def classification_performance_evaluation(y_true, y_pred, print_output = False):
  '''
  Accepts data of the ground truth labels and the predicted labels
  Returns a series of different classification scores
  If print_output = True it will print the scores as well as returning them
  '''
  #Both homogeneity and completeness have values 0 to 1. 
  #Larger values being preferable
  #V measure is a 'harmonic' mean of homogeneity and completeness
  h,c,v = homogeneity_completeness_v_measure(y_true, y_pred)
    
  #accuracy score displays % of labels correctly labelled.
  acc_score = accuracy_score(y_true, y_pred)
    
  #balanced accuracy to adjust for inbalanced classes
  bal_acc_score = balanced_accuracy_score(y_true, y_pred)
    
  #f1_score
  #Harmonic mean of precision and recall
  #Values of 0 to 1. 1 is best.
  score_f1 = f1_score(y_true, y_pred)
    
  if print_output == True:
      print(f"Homogeneity Score: {h:.3f}\
      \nCompleteness Score: {c:.3f}\
      \nV_measure Score: {v:.3f}\
      \nAccuracy Score: {acc_score:.3f}\
      \nBalanced Accuracy Score: {bal_acc_score:.3f}\
      \nF1 Score: {score_f1:.3f}")

  if print_output == False:
      return [h, c, v, acc_score, bal_acc_score, score_f1]

def classification_using_clustering_performance_evaluation(y_true, y_pred, print_output = False, classification = True):
  '''
  Accepts data of the ground truth labels and the predicted labels
  Returns a series of different clustering and classification scores
  If print_output = True it will print the scores as well as returning them
  If classification = False will correct accuracy metrics (because clustering does not ascribe which cluster is which label)
  '''
  #Both homogeneity and completeness have values 0 to 1. 
  #Larger values being preferable
  #V measure is a 'harmonic' mean of homogeneity and completeness
  h,c,v = homogeneity_completeness_v_measure(y_true, y_pred)
    
  #accuracy score displays % of labels correctly labelled.
  acc_score = accuracy_score(y_true, y_pred)
    
  #balanced accuracy to adjust for inbalanced classes
  bal_acc_score = balanced_accuracy_score(y_true, y_pred)
    
  #f1_score
  #Harmonic mean of precision and recall
  #Values of 0 to 1. 1 is best.
  score_f1 = f1_score(y_true, y_pred)

  #Adjusted rand score
  #Similarity between clusters by counting all pairs of clusters,
  #Higher values when greater proportion is in correct cluster
  adj_rand_score = adjusted_rand_score(y_true, y_pred)
    
  #if a clustering method selected that did not have information about labels select classification as False
  if classification == False:
      #Must take maximum of 1-score and score because clustering method is only labelling two groups without
      #assigning which is which
      acc_score = max(acc_score, (1-acc_score))
      bal_acc_score = max(bal_acc_score, (1-bal_acc_score))
      score_f1 = max(score_f1, (1-score_f1))

      if print_output == True:
        print(f"Homogeneity Score: {h:.3f}\
        \nCompleteness Score: {c:.3f}\
        \nV_measure Score: {v:.3f}\
        \nAccuracy Score: {acc_score:.3f}\
        \nBalanced Accuracy Score: {bal_acc_score:.3f}\
        \nF1 Score: {score_f1:.3f}\
        \nAdjusted_rand_score : {adj_rand_score:.3f}")
      return [h, c, v, acc_score, bal_acc_score, score_f1, adj_rand_score]
  
  
  elif classification == True: 
    if print_output == True:
      print(f"Homogeneity Score: {h:.3f}\
      \nCompleteness Score: {c:.3f}\
      \nV_measure Score: {v:.3f}\
      \nAccuracy Score: {acc_score:.3f}\
      \nBalanced Accuracy Score: {bal_acc_score:.3f}\
      \nF1 Score: {score_f1:.3f}\
      \nAdjusted_rand_score : {adj_rand_score:.3f}")
    
  if print_output == False:
      return [h, c, v, acc_score, bal_acc_score, score_f1, adj_rand_score]
    
def plot_precision_recall(y_true, y_pred):
  '''
  Accepts ground truth and prediction labels
  Plots a precision recall graph
  '''
  #Plots the precision_recall_graph
  PrecisionRecallDisplay.from_predictions(y_true, y_pred)
  plt.ylim(0,1)
  plt.show()

def plot_roc_and_print_auc(y_true, y_pred):
  '''
  Accepts ground truth and prediction labels
  Plots a ROC curve with AUC calculated value
  '''
  #Plots the Receiver Operating Characteristic
  #And prints the ROC_AUC (area under curve) value
  #Values of 0 to 1. 1 is best
  false_positive_rate, true_positive_rate, threshold  =  roc_curve(y_true, y_pred)
  area_under_curve = round(roc_auc_score(y_true, y_pred), 3)
  zero_to_one_array = np.linspace(0,1,100)
    
  #plot x = y graph
  plt.plot(zero_to_one_array, zero_to_one_array, linestyle = 'dashed', c = 'k')
  plt.plot(false_positive_rate, true_positive_rate, label = (f"Area Under Curve:\n{area_under_curve}"))
  plt.legend(loc = 'lower right')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('ROC Curve and AUC Score')
  plt.show()
    
def plot_confusion_matrix(y_true, y_pred):    
  ConfusionMatrixDisplay.from_predictions(y_true, y_pred)
  plt.show()

## Loading The Data

In [152]:
#Load data as json
bcd = datasets.load_breast_cancer()
#Load data as frames
# X, y = datasets.load_breast_cancer(return_X_y = True, as_frame = True)
#Convert frames to single dataframe
# df = pd.concat([X, y], axis =1)

In [153]:
X = bcd.data

In [154]:
y = bcd.target

In [155]:
X_columns = bcd.feature_names

In [156]:
#Print first 3 rows
print(X_columns)
for i in range(3):
  print(X[i])

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
[2.057e+01 1.777e+01 1.329e+02 1.326e+03 8.474e-02 7.864e-02 8.690e-02
 7.017e-02 1.812e-01 5.667e-02 5.435e-01 7.339e-01 3.398e+00 7.408e+

## Feature Engineering
Many of these features relate to the same aspect of the data.

Therefore there is extra information that can be extracted.

To do this we will create retain the feaures so far and create a variety of new derived features

Coding inspired and expanded from [2]

[2] Python Feature Engineering Cookbook - Packt 2020, Chapter 9 - Applying Mathematical Computations to Features

In [157]:
#create new 'difference' columns
original_col_length = len(X_columns)
for i in range(original_col_length - 20):
    new_col_name = str(''.join([X_columns[i+20][:5], ' - ',X_columns[i]]))
    print(new_col_name)
    X_columns = np.append(X_columns,new_col_name) 
    new_column_data = X[:,[i+20]] - X[:,[i]]
    X = np.concatenate((X, new_column_data), axis = 1)

worst - mean radius
worst - mean texture
worst - mean perimeter
worst - mean area
worst - mean smoothness
worst - mean compactness
worst - mean concavity
worst - mean concave points
worst - mean symmetry
worst - mean fractal dimension


In [158]:
#create new 'ratio' columns
for i in range(original_col_length - 20):
    new_col_name = str(''.join(['ratio of ', X_columns[i+20][:5].strip(), ' to ',X_columns[i]]))
    print(new_col_name)
    X_columns = np.append(X_columns,new_col_name) 
    new_column_data = X[:,[i+20]] / X[:,[i]]
    X = np.concatenate((X, new_column_data), axis = 1)

ratio of worst to mean radius
ratio of worst to mean texture
ratio of worst to mean perimeter
ratio of worst to mean area
ratio of worst to mean smoothness
ratio of worst to mean compactness
ratio of worst to mean concavity
ratio of worst to mean concave points
ratio of worst to mean symmetry
ratio of worst to mean fractal dimension


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in true_divide
  


Error message given as we have generated some nan values:

In [159]:
np.argwhere(np.isnan(X))

array([[101,  46],
       [101,  47],
       [140,  46],
       [140,  47],
       [174,  46],
       [174,  47],
       [175,  46],
       [175,  47],
       [192,  46],
       [192,  47],
       [314,  46],
       [314,  47],
       [391,  46],
       [391,  47],
       [473,  46],
       [473,  47],
       [538,  46],
       [538,  47],
       [550,  46],
       [550,  47],
       [557,  46],
       [557,  47],
       [561,  46],
       [561,  47],
       [568,  46],
       [568,  47]])

We shall deal with these after the pre-processing scaling

We can also create features from distribution of each of the mean, error and worst features respectively.

In [160]:
# Mean error and worst features are in 

# mean_features = list(X.columns[:10])
# error_features = list(X.columns[10:20])
# worst_features = list(X.columns[20:30])
   
X_columns = np.append(X_columns, ['min_of_mean_features', 'mean_of_mean_features', 'max_of_mean_features',
                                  'std_of_mean_features', 'multiple_of_mean_features', 'sum_of_mean_features',
                                  'min_of_error_features', 'mean_of_error_features', 'max_of_error_features', 
                                  'std_of_error_features', 'multiple_of_error_features', 'sum_of_error_features', 
                                  'min_of_worst_features', 'mean_of_worst_features', 'max_of_worst_features', 
                                  'std_of_worst_features', 'multiple_of_worst_features', 'sum_of_worst_features']) 

for feature_ranges_data in (X[:,0:10], X[:,10:20], X[:,20:30]):
  min_data = np.amin(feature_ranges_data, axis = 1).reshape(569,1)
  mean_data = np.mean(feature_ranges_data, axis = 1).reshape(569,1)
  max_data = np.amax(feature_ranges_data, axis = 1).reshape(569,1)
  std_data = np.std(feature_ranges_data, axis = 1).reshape(569,1)
  multiple_data = np.prod(feature_ranges_data, axis = 1).reshape(569,1)
  sum_data = np.sum(feature_ranges_data, axis = 1).reshape(569,1)
  new_data = np.concatenate((min_data, mean_data , max_data, std_data, multiple_data, sum_data), axis = 1)
  X = np.concatenate((X, new_data), axis = 1)

In [161]:
print(f" The new number of feature columns is now {len(X_columns)}")
print(f" The shape of X data is now {X.shape[0]} rows by {X.shape[1]} columns")

 The new number of feature columns is now 68
 The shape of X data is now 569 rows by 68 columns


In [162]:
#Combine X and y into complete 2d array for easier filtering.
data_complete = np.concatenate((X, y.reshape(569,1)), axis = 1)
print(f" The shape of the complete dataset is {data_complete.shape[0]} rows by {data_complete.shape[1]} columns")

 The shape of the complete dataset is 569 rows by 69 columns


In [163]:
#create two sub arrays to contain the malignant and benign data
malignant = data_complete[data_complete[:,-1] == 0]
benign = data_complete[data_complete[:,-1] == 1]
print(f"The shape of the malignant dataset is {malignant.shape[0]} rows by {malignant.shape[1]} columns")
print(f"The shape of the benign dataset is {benign.shape[0]} rows by {benign.shape[1]} columns")
print(f"These match the values seen in the description:\n{(bcd.DESCR[3085:3140])}")

The shape of the malignant dataset is 212 rows by 69 columns
The shape of the benign dataset is 357 rows by 69 columns
These match the values seen in the description:
Class Distribution: 212 - Malignant, 357 - Benign

    


In [ ]:
#Show distribution of each feature as boxplots.
fig = plt.figure(figsize = (15,80))
for index, column_name in enumerate(X_columns):
    ax = fig.add_subplot(23,3,index + 1)
    plt.boxplot(x = X[:,index])
    plt.title(column_name)
    plt.xticks([])
plt.tight_layout()

In [ ]:
#Show distribution overlap of each sub-dataframe (benign and malignant) feature as kde (kernel density plots).
#Requires temporary pandas dataframes for kde:

temp_df = pd.DataFrame(data_complete, columns = np.append(X_columns ,'target'))
temp_malignant = temp_df[temp_df['target'] == 0]
temp_benign = temp_df[temp_df['target'] == 1]
fig = plt.figure(figsize = (15,80))
for index, column_name in enumerate(X_columns):
    ax = fig.add_subplot(23,3,index + 1)
    temp_malignant[column_name].plot.kde(ax = ax, label = 'Malignant')
    temp_benign[column_name].plot.kde(ax = ax, label = 'Benign')
    plt.title(column_name)
    if index == 0:
        plt.legend()
plt.tight_layout()

In [ ]:
#Linear correlations with the target
# Positive correlation values mean a greater positive value in the column is more likely to be benign
# Negative correlation values mean a greater positive value in the column is more likely to be malignant

for index, row in (temp_df.corr()['target'].sort_values().drop('target').iteritems()):
  index_str_length = len(index)
  spacing = 40 - index_str_length
  if row >= 0:
    print(f"{index} {' '*(spacing+1)} {row:.3f}")
  else:
    print(f"{index} {' '*(spacing)} {row:.3f}")

In [ ]:
#create a list to place similar columns together (used for plotting purposes)
similar_column_ordered = []
for i in range(10):
    similar_column_ordered.append(X_columns[i])
    similar_column_ordered.append(X_columns[i+10])
    similar_column_ordered.append(X_columns[i+20])

In [ ]:
#Print distribution of benign and malignant in original feature columns
fig = plt.figure(figsize = (15,60))
for index, column_name in enumerate(similar_column_ordered):
    ax = fig.add_subplot(10,3,index + 1)
    plt.scatter(x = temp_benign.index, y = temp_benign[column_name], label = 'Benign', alpha = 0.1)
    plt.scatter(x = temp_malignant.index, y = temp_malignant[column_name], label = 'Malignant', alpha = 0.1)
    plt.xticks([])
    plt.ylabel(f"{column_name} value")
    plt.title(f"{column_name}")
    if index == 0:
        plt.legend()
        plt.xlabel(f'Data given false X_values to \n show points more clearly')
        

In [ ]:
#Print distribution of benign and malignant in new feature columns
fig = plt.figure(figsize = (15,100))
for index, column_name in enumerate(X_columns[30:]):
    ax = fig.add_subplot(20,3,index + 1)
    plt.scatter(x = temp_benign.index, y = temp_benign[column_name], label = 'Benign', alpha = 0.1)
    plt.scatter(x = temp_malignant.index, y = temp_malignant[column_name], label = 'Malignant', alpha = 0.1)
    plt.xticks([])
    plt.ylabel(f"{column_name} value")
    plt.title(f"{column_name}")
    if index == 0:
        plt.legend()
        plt.xlabel(f'Data given false X_values to \n show points more clearly')
plt.tight_layout()

In [ ]:
#For comparing 2 variables calculate number of permutations:
n_factorial = np.math.factorial(len(X_columns))
n_minus_r_factorial = np.math.factorial(len(X_columns) - 2)
print(f"We have {len(X_columns)} columns.")
print(f"For Comparing 2 Variables there are {int(n_factorial/ n_minus_r_factorial)} permutations")

4556 permutations means there are 4556 unique ways to compare 2 of the 68 variables.

We could manually print all comparisons in scatter graphs and visually pick the features that give the best separation

A more powerful method is too loop through 68 permutations (as there are 68 feature columns) at a time using a column distance to ensure no repetition of permutations.

The first run will compare columns next to each other (1 column distance)

The second run will compare columns two away from each other (2 column distance)

And so on. Then we can generate a score based on a simple K Nearest Neighbours method. The best separation will have the highest accuracy scores

In [ ]:
'''
temp_X_df = pd.DataFrame(X, columns = X_columns))
temp_y_df = pd.DataFrame(y, columns = 'Target')
seen_comparisons = []
two_variable_comparisons = pd.DataFrame({
    'variable_1': [],
    'variable_2': [],
    'accuracy_score': []})
column_length = len(X_columns)
for column_distance in np.arange(1, column_length +1):

    for index, column_name in enumerate(X_columns):
        comparison_index = index + column_distance
        if (comparison_index) >= column_length:
            comparison_index = (comparison_index) - column_length
        KNN_model = KNeighborsClassifier()
        variable_1 = X_columns[index]
        variable_2 = X_columns[comparison_index]
        KNN_model.fit(temp_X_df[[variable_1, variable_2]], temp_y_df)
        score = KNN_model.score(temp_X_df[[variable_1, variable_2]], temp_y_df)
        
        #Only add combination if its not already been seen.
        if [variable_2, variable_1] not in seen_comparisons:
            seen_comparisons.append([variable_1, variable_2])
            two_variable_comparisons = two_variable_comparisons.append({'variable_1': variable_1,
                                                                        'variable_2': variable_2,
                                                                        'accuracy_score': score},
                                                                       ignore_index = True)
top_30 = two_variable_comparisons.sort_values(by = 'accuracy_score', ascending = False)[:30]
top_30 = top_30.reset_index(drop=True)
'''
#Note the code above takes significant run time.
#For simplicity it was saved as below to github page 
#and will be loaded from there
'''
top_30.to_csv('top_30.csv')
'''

In [ ]:
top_30 = pd.read_csv('https://raw.githubusercontent.com/adbandy/data_files/main/top_30.csv', index_col = 0)
top_30

In [ ]:
#Visualise these top combinations
fig = plt.figure(figsize = (15,60))
for index in np.arange(len(top_30)):
    variable_1 = top_30['variable_1'].iloc[index]
    variable_2 = top_30['variable_2'].iloc[index]
    score = top_30['accuracy_score'].iloc[index]
    
    ax = fig.add_subplot(10,3,index + 1)
    plt.scatter(x = temp_malignant[variable_1], y = temp_malignant[variable_2], label = 'Malignant', alpha = 0.1)
    plt.scatter(x = temp_benign[variable_1], y = temp_benign[variable_2], label = 'Benign', alpha = 0.1)
    plt.xlabel(variable_1)
    plt.ylabel(variable_2)
    plt.title(f"Score: {score:.4f}")
    if index == 0:
        plt.legend()


Clearly our newly created features will have value in separation of clusters.

## Feature Engineering - Standardisation

First we will split the data into a training set and testing set in an (80:20 ratio)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"The % proportions of classes in the splits are:")
print(f"Training benign: {((len(y_train[y_train == 1]) / len(y_train)) * 100):.2f}%")
print(f"Training malignant: {((len(y_train[y_train == 0]) / len(y_train)) * 100):.2f}%\n")
print(f"Testing benign: {((len(y_test[y_test == 1]) / len(y_test)) * 100):.2f}%")
print(f"Testing malignant: {((len(y_test[y_test == 0]) / len(y_test)) * 100):.2f}%")

# \ny_train :\n{round(y_train.value_counts(normalize=True)*100,2)}\n\ny_test:\n{round(y_test.value_counts(normalize=True)*100,2)}")

As we can see our split is sufficiently stratified.


We must use X_train to fit a scaler. Then rescale the X_train and X_test. This is done to give each column equal importance in the modelling. 


In [ ]:
columns = X_columns
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train = pd.DataFrame(X_train, columns = columns)
X_test = pd.DataFrame(X_test, columns = columns)
#Seperate scaling required for clustering as train test is meaningless here
full_scaler = StandardScaler().fit(X)
X = full_scaler.transform(X)
#Display a scaled preview. 
pd.DataFrame(X, columns = columns).head()

In [ ]:
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))
#print(type(X_train_pca))
#print(type(X_test_pca))
#print(type(X_pca))
#print(type(X_pca))

In [ ]:
#Check for nan values generated during standardisation
pd.DataFrame(X, columns = columns).isna().sum().sort_values(ascending = False)

As discussed earlier these values are errors generated as numpy was unable to store values so small in float 64. Having standardised the data we can safely replace these with zeros.

In [ ]:
X = np.nan_to_num(X, nan = 0.0)
X_train = np.nan_to_num(X_train, nan = 0.0)
X_test = np.nan_to_num(X_test, nan = 0.0)

In [ ]:
#Simple check to show this worked.
pd.DataFrame(X, columns = columns).isna().sum().sort_values(ascending = False)

In [ ]:
#repeat for X_test:
pd.DataFrame(X_test, columns = columns).isna().sum().sort_values(ascending = False)

In [ ]:
#repeat for X_train:
pd.DataFrame(X_train, columns = columns).isna().sum().sort_values(ascending = False)

## Feature Engineering: Dimensionallity Reduction - Using PCA

Principle Component Analysis is a technique used to transform the data and determine new features that explain variance more succinctly. 

The mathematics is complex. The aim is to be able to reduce the feature space significantly and avoid the 'curse of dimensionality'. 

In essence as the number of features increases, the amount of data required to sufficiently explain it can grow exponentially.

In [ ]:
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))
#print(type(X_train_pca))
#print(type(X_test_pca))

In [ ]:
#Instantiate the PCA and fit using the training data.
pca = PCA()
pca.fit(X_train)

#Then transform the train and test X using this fitted PCA 
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
pca_var_perc = pca.explained_variance_ratio_ * 100

# PCA all data
pca = PCA()
pca.fit(X)
X_pca = pca.transform(X)

#plot variance explained for each new feature 
fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(10,8))
ax.bar(x=[i for i in range(len(pca_var_perc))],height=pca_var_perc)
plt.xlabel("Number Of Principal Components Used")
plt.ylabel("Percent Explained Variance")
plt.yticks(np.arange(0,50,5))
plt.show()

In [ ]:
#Display same information in value format
pca_explained_variance = pd.DataFrame(pca.explained_variance_ratio_).head(15)
pca_explained_variance.columns = ['% Of Explained Variance In Each Column']
pca_explained_variance.apply(lambda x: round(x,3))

In [ ]:
#Store as dataframe for plotting purposes
pca_var_perc_df = pd.DataFrame(X_train_pca)
pca_var_perc_df['target'] = y_train
pca_var_perc_df.head()

In [ ]:
#Plot distribution of data for two PCA variable x,y graphs.

sub_df_benign = pca_var_perc_df[pca_var_perc_df['target'] == 1] 
sub_df_malignant = pca_var_perc_df[pca_var_perc_df['target'] == 0]

fig = plt.figure(figsize = (10,30))
for i in range(5):
  for j in  range(5):
    if (i == 0) & (j == 0):
      fig.add_subplot(6,5, 1)
      plt.xlabel(f"Principal Component 0")
      plt.ylabel(f"Principal Component 1")

      plt.scatter(sub_df_benign[sub_df_benign.columns[i]], sub_df_benign[sub_df_benign.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant[sub_df_malignant.columns[i]], sub_df_malignant[sub_df_malignant.columns[j+1]], alpha = 0.1, label = 'Malignant')
      plt.legend()
    #Remove duplicate graphs (i.e Component 1 vs 0 is same as Component 0 vs 1)
    elif i > (j):
      pass
    else:
      fig.add_subplot(6,5, ((i*5)+(j+1)))
      plt.xlabel(f"Principal Component {i}")
      plt.ylabel(f"Principal Component {j+1}")

      plt.scatter(sub_df_benign[sub_df_benign.columns[i]], sub_df_benign[sub_df_benign.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant[sub_df_malignant.columns[i]], sub_df_malignant[sub_df_malignant.columns[j+1]], alpha = 0.1, label = 'Malignant')

plt.tight_layout()

## Feature Engineering - Dimensionality Reduction- K-PCA

Kernel PCA is another similar technique compared to PCA but can work better on more complex relationships.

In [ ]:
#Kernel PCA, Instantiate, fit and transform data.

k_pca = KernelPCA()
k_pca.fit(X_train)
X_train_k_pca = k_pca.transform(X_train)
X_test_k_pca = k_pca.transform(X_test)

In [ ]:
#Store as dataframe for plotting purposes
k_pca_var_perc_df = pd.DataFrame(X_train_k_pca)
k_pca_var_perc_df['target'] = y_train
k_pca_var_perc_df.head()

In [ ]:
#Plot distribution of data for two K-PCA variable x,y graphs.


sub_df_benign = k_pca_var_perc_df[k_pca_var_perc_df['target'] == 1] 
sub_df_malignant = k_pca_var_perc_df[k_pca_var_perc_df['target'] == 0]

fig = plt.figure(figsize = (10,30))
for i in range(5):
  for j in  range(5):
    if (i == 0) & (j == 0):
      fig.add_subplot(6,5, 1)
      plt.xlabel(f"Principal Component 0")
      plt.ylabel(f"Principal Component 1")

      plt.scatter(sub_df_benign[sub_df_benign.columns[i]], sub_df_benign[sub_df_benign.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant[sub_df_malignant.columns[i]], sub_df_malignant[sub_df_malignant.columns[j+1]], alpha = 0.1, label = 'Malignant')
      plt.legend()
    #Remove duplicate graphs (i.e Component 1 vs 0 is same as Component 0 vs 1)
    elif i > (j):
      pass
    else:
      fig.add_subplot(6,5, ((i*5)+(j+1)))
      plt.xlabel(f"Principal Component {i}")
      plt.ylabel(f"Principal Component {j+1}")

      plt.scatter(sub_df_benign[sub_df_benign.columns[i]], sub_df_benign[sub_df_benign.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant[sub_df_malignant.columns[i]], sub_df_malignant[sub_df_malignant.columns[j+1]], alpha = 0.1, label = 'Malignant')

plt.tight_layout()

## Feature Engineering - Comparing PCA with K_PCA
Visually these two methods appear to show similar results.

Below is a replot of the first rows from each graphic.

In [ ]:
#Plot first row of each graphic (PCA and K-PCA)

sub_df_benign_pca = k_pca_var_perc_df[k_pca_var_perc_df['target'] == 1] 
sub_df_malignant_pca = k_pca_var_perc_df[k_pca_var_perc_df['target'] == 0]

sub_df_benign_k_pca = k_pca_var_perc_df[k_pca_var_perc_df['target'] == 1] 
sub_df_malignant_k_pca = k_pca_var_perc_df[k_pca_var_perc_df['target'] == 0]

fig = plt.figure(figsize = (10,30))

for i in [0]:
  for j in  range(5):
    if (i == 0) & (j == 0):
      fig.add_subplot(6,5, 1)
      plt.xlabel(f"PCA Component 0")
      plt.ylabel(f"PCA Component 1")

      plt.scatter(sub_df_benign_pca[sub_df_benign_pca.columns[i]], sub_df_benign_pca[sub_df_benign_pca.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant_pca[sub_df_malignant_pca.columns[i]], sub_df_malignant_pca[sub_df_malignant_pca.columns[j+1]], alpha = 0.1, label = 'Malignant')
      plt.legend()
    else:
      fig.add_subplot(6,5, ((i*5)+(j+1)))
      plt.xlabel(f"PCA Component {i}")
      plt.ylabel(f"PCA Component {j+1}")

      plt.scatter(sub_df_benign_pca[sub_df_benign_pca.columns[i]], sub_df_benign_pca[sub_df_benign_pca.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant_pca[sub_df_malignant_pca.columns[i]], sub_df_malignant_pca[sub_df_malignant_pca.columns[j+1]], alpha = 0.1, label = 'Malignant')     


for i in [0]:
  for j in  range(5):
    if (i == 0) & (j == 0):
      fig.add_subplot(6,5, 6)
      plt.xlabel(f"K_PCA Component 0")
      plt.ylabel(f"K_PCA Component 1")

      plt.scatter(sub_df_benign_k_pca[sub_df_benign_k_pca.columns[i]], sub_df_benign_k_pca[sub_df_benign_k_pca.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant_k_pca[sub_df_malignant_k_pca.columns[i]], sub_df_malignant_k_pca[sub_df_malignant.columns[j+1]], alpha = 0.1, label = 'Malignant')
      plt.legend()
    else:
      fig.add_subplot(6,5, (5+j+1))
      plt.xlabel(f"K_PCA Component {i}")
      plt.ylabel(f"K_PCA Component {j+1}")

      plt.scatter(sub_df_benign_k_pca[sub_df_benign_k_pca.columns[i]], sub_df_benign_k_pca[sub_df_benign_k_pca.columns[j+1]], alpha = 0.1, label = 'Benign')
      plt.scatter(sub_df_malignant_k_pca[sub_df_malignant_k_pca.columns[i]], sub_df_malignant_k_pca[sub_df_malignant.columns[j+1]], alpha = 0.1, label = 'Malignant')

plt.tight_layout()

No discernable differences between PCA and k_PCA

In [ ]:
#Having completed feature engineering we can also store X as a dataframe for plotting purposes later.
X = pd.DataFrame(X, columns = X_columns)
y = pd.Series(y)

In [ ]:
#And store train test splits as dataframes for plotting purposes later.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(f"The % proportions of classes in the splits are:")
print(f"Training benign: {((len(y_train[y_train == 1]) / len(y_train)) * 100):.2f}%")
print(f"Training malignant: {((len(y_train[y_train == 0]) / len(y_train)) * 100):.2f}%\n")
print(f"Testing benign: {((len(y_test[y_test == 1]) / len(y_test)) * 100):.2f}%")
print(f"Testing malignant: {((len(y_test[y_test == 0]) / len(y_test)) * 100):.2f}%")

In [ ]:
print(type(X))
print(type(X_pca))

print(type(X_train))
print(type(X_test))
print(type(X_pca))
print(type(y))

In [ ]:
X_all = X.to_numpy()
y_all = y.to_numpy()

X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [ ]:
print(type(X_all))
print(type(X_pca))

print(type(X_train_np))
print(type(X_test_np))

print(type(X_pca))
print(type(y_all))

print(type(X_train_pca))
print(type(X_test_pca))

# Part II Application: Clustering

## Spectral Clustering
In general this technique is most suitable for when the data does not form seperate spherical like clusters with little overlap.
For instance it works well for sphere within a hollow sphere or double helix style structure.

Whether this method is applicable is difficult for multi-dimensional space, but from seeing our top_30 comparisons (of one variable vs another) we can say some combinations are potentially spherical-like and separable.

First let us review a simplistic model. And test it

In [ ]:
spectral_cluster_model = SpectralClustering(n_clusters = 2, random_state = 1, n_init = 10, affinity = 'nearest_neighbors', assign_labels= 'kmeans')
spectral_cluster_model.fit(X_all)

In [ ]:
print(type(X_all))
print(type(X_pca))

print(type(X_train_np))
print(type(X_test_np))

print(type(X_pca))
print(type(y_all))

print(type(X_train_pca))
print(type(X_test_pca))

In [ ]:
clustering_performance_evaluation(X_all, spectral_cluster_model.labels_, print_output = True)

High Score is desirable for Calinski-Harabasz Score.
The best Silhouette score a model can obtain is 1 with 0 being the worst.

In [ ]:
simple_scores_spectral_clustering = classification_using_clustering_performance_evaluation(y, spectral_cluster_model.labels_, print_output = True, classification= False)

In all of these classifications scores, higher values are better.

### Spectral Clustering - Hyperparameter Tuning

In [ ]:
#Create a dataframe for storing scores and parameters used.
spectral_clustering_hyperparameter_tuning_df = pd.DataFrame({'n_init': [],
                                                             'eigen_solver': [],
                                                             'n_neigbors': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

In [ ]:
for n_init in [10,100,1000]:
    for eigen_solver in ['arpack', 'lobpcg']:
        for n_neigbors in [1,10,100]:
            spectral_cluster_model= SpectralClustering(n_clusters = 2,
                                                       random_state = 1,
                                                       affinity = 'nearest_neighbors',
                                                       assign_labels= 'kmeans',
                                                       n_init = n_init,
                                                      eigen_solver= eigen_solver,
                                                      n_neighbors= n_neigbors)
            spectral_cluster_model.fit(X_all)
            h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, spectral_cluster_model.labels_ , classification= False)
            ss, chs, dbs =  clustering_performance_evaluation(X_all, spectral_cluster_model.labels_)
            spectral_clustering_hyperparameter_tuning_df = \
                spectral_clustering_hyperparameter_tuning_df.append({'n_init': n_init,
                                                                     'eigen_solver': eigen_solver,
                                                                     'n_neigbors': n_neigbors,
                                                                     'Homogeneity Score': h,
                                                                     'Completeness Score': c,
                                                                     'V_measure Score': v,
                                                                     'Accuracy Score': a,
                                                                     'Balanced Accuracy Score': b,
                                                                     'F1 Score': f1,
                                                                     'Adjusted_rand_score': ars,
                                                                     'Silhouette Score': ss,
                                                                     'Calinski-Harabasz Score': chs,
                                                                     'David-Bouldin Score': dbs}, ignore_index = True)                                                             

In [ ]:
spectral_clustering_hyperparameter_tuning_df.sort_values(by = 'Adjusted_rand_score', ascending = False)

Higher n_neigbhours is most influential here. Will experiment with high neighbour count. Other hyperparameter tuning has had little effect.

In [ ]:
#Create a dataframe for storing scores and parameters used.
spectral_clustering_hyperparameter_tuning_n_neighbours_df = pd.DataFrame({'n_init': [],
                                                             'eigen_solver': [],
                                                             'n_neigbors': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

In [ ]:
for n_init in [10]:
    for eigen_solver in ['arpack']:
        
        for n_neigbors in list(np.arange(50,550, 10)):
            spectral_cluster_model= SpectralClustering(n_clusters = 2,
                                                       random_state = 1,
                                                       affinity = 'nearest_neighbors',
                                                       assign_labels= 'kmeans',
                                                       n_init = n_init,
                                                      eigen_solver= eigen_solver,
                                                      n_neighbors= n_neigbors)
            spectral_cluster_model.fit(X_all)
            h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, spectral_cluster_model.labels_, classification= False)
            ss, chs, dbs =  clustering_performance_evaluation(X_all, spectral_cluster_model.labels_)
            spectral_clustering_hyperparameter_tuning_n_neighbours_df = \
                spectral_clustering_hyperparameter_tuning_n_neighbours_df.append({'n_init': n_init,
                                                                     'eigen_solver': eigen_solver,
                                                                     'n_neigbors': n_neigbors,
                                                                     'Homogeneity Score': h,
                                                                     'Completeness Score': c,
                                                                     'V_measure Score': v,
                                                                     'Accuracy Score': a,
                                                                     'Balanced Accuracy Score': b,
                                                                     'F1 Score': f1,
                                                                     'Adjusted_rand_score': ars,
                                                                     'Silhouette Score': ss,
                                                                     'Calinski-Harabasz Score': chs,
                                                                     'David-Bouldin Score': dbs}, ignore_index = True)                                                             

In [ ]:
spectral_clustering_hyperparameter_tuning_n_neighbours_df = spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by = 'Adjusted_rand_score', ascending = False)
spectral_clustering_hyperparameter_tuning_n_neighbours_df[['Homogeneity Score',
       'Completeness Score', 'V_measure Score', 'Accuracy Score',
       'Balanced Accuracy Score', 'F1 Score', 'Adjusted_rand_score']] = spectral_clustering_hyperparameter_tuning_n_neighbours_df[['Homogeneity Score',
       'Completeness Score', 'V_measure Score', 'Accuracy Score',
       'Balanced Accuracy Score', 'F1 Score', 'Adjusted_rand_score']].apply(lambda x: round(x,3))
spectral_clustering_hyperparameter_tuning_n_neighbours_df.head(5)

In [ ]:
spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by = 'Adjusted_rand_score', ascending = False)

In [ ]:
#Sort by n_neighbours for plotting purposes
spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted = spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'n_neigbors')

In [ ]:
#plot the various scores
fig = plt.figure(figsize = (10,40))
for i,j in zip(range(1,11), ['Homogeneity Score',
       'Completeness Score', 'V_measure Score', 'Accuracy Score',
       'Balanced Accuracy Score', 'F1 Score', 'Adjusted_rand_score',
       'Silhouette Score', 'Calinski-Harabasz Score', 'David-Bouldin Score']):
    fig.add_subplot(5,2,i)
    plt.plot(spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted['n_neigbors'],
            spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])
    plt.title(f"{j}\n Vs Number Of Neigbors Clustering")
    plt.xlabel(f"Number Of Neigbors")
    plt.ylabel(f"{j}")
    if i <= 7:
        plt.axvline(130, linestyle = 'dashed', alpha = 0.2)
        y_value_for_text = ((np.max(spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j]) - np.min(spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])) / 2) + np.min(spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])
        plt.text(140,y_value_for_text, s='130', rotation = 90, c= '#1f77b4', alpha = 0.5)
    if i >7:
        plt.axvline(540, linestyle = 'dashed', alpha = 0.2)
        y_value_for_text = ((np.max(spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j]) - np.min(spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])) / 2) + np.min(spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])
        plt.text(520,y_value_for_text, s='540', rotation = 90, c= '#1f77b4', alpha = 0.5)


Best overall classification metrics are given at 130 Neighbours. 

In [ ]:
best_score_ss = spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'Silhouette Score', ascending = False).iloc[0]['n_neigbors']
best_score_chs = spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'Calinski-Harabasz Score', ascending = False).iloc[0]['n_neigbors']
best_score_dbs = spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'David-Bouldin Score', ascending = True).iloc[0]['n_neigbors']

print(f"N_neigbors that gave the best clustering metrics: \
      \nFor Silhouette Score : {int(best_score_ss)} \
      \nFor Calinski-Harabasz Score : {int(best_score_chs)} \
      \nFor David-Bouldin Score : {int(best_score_dbs)}")

Next lets compare these values with those obtained using the PCA reduced dimensionality data

We will use only the top 7 columns from PCA

In [ ]:
X_pca = X_pca[:,:6]

In [ ]:
#Create a dataframe for storing scores and parameters used.
pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df = pd.DataFrame({'n_init': [],
                                                             'eigen_solver': [],
                                                             'n_neigbors': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

In [ ]:
for n_init in [10]:
    for eigen_solver in ['arpack']:
        
        for n_neigbors in list(np.arange(50,550, 10)):
            pca_spectral_cluster_model= SpectralClustering(n_clusters = 2,
                                                       random_state = 1,
                                                       affinity = 'nearest_neighbors',
                                                       assign_labels= 'kmeans',
                                                       n_init = n_init,
                                                      eigen_solver= eigen_solver,
                                                      n_neighbors= n_neigbors)
            pca_spectral_cluster_model.fit(X_pca[:,:6])
            h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, pca_spectral_cluster_model.labels_, classification= False)
            ss, chs, dbs =  clustering_performance_evaluation(X_all, pca_spectral_cluster_model.labels_)
            pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df = \
                pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df.append({'n_init': n_init,
                                                                     'eigen_solver': eigen_solver,
                                                                     'n_neigbors': n_neigbors,
                                                                     'Homogeneity Score': h,
                                                                     'Completeness Score': c,
                                                                     'V_measure Score': v,
                                                                     'Accuracy Score': a,
                                                                     'Balanced Accuracy Score': b,
                                                                     'F1 Score': f1,
                                                                     'Adjusted_rand_score': ars,
                                                                     'Silhouette Score': ss,
                                                                     'Calinski-Harabasz Score': chs,
                                                                     'David-Bouldin Score': dbs}, ignore_index = True)

In [ ]:
pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'Adjusted_rand_score', ascending = False )

In [ ]:
pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted = pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'n_neigbors')
fig = plt.figure(figsize = (10,40))
for i,j in zip(range(1,11), ['Homogeneity Score',
       'Completeness Score', 'V_measure Score', 'Accuracy Score',
       'Balanced Accuracy Score', 'F1 Score', 'Adjusted_rand_score',
       'Silhouette Score', 'Calinski-Harabasz Score', 'David-Bouldin Score']):
    fig.add_subplot(5,2,i)
    plt.plot(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted['n_neigbors'],
            pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])
    plt.title(f"{j}\n Vs Number Of Neigbors Clustering")
    plt.xlabel(f"Number Of Neigbors")
    plt.ylabel(f"{j}")
    if i <= 7:
        plt.axvline(140, linestyle = 'dashed', alpha = 0.2)
        y_value_for_text = ((np.max(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j]) - np.min(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])) / 2) + np.min(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])
        plt.text(150,y_value_for_text, s='140', rotation = 90, c= '#1f77b4', alpha = 0.5)
    if i >7:
        plt.axvline(540, linestyle = 'dashed', alpha = 0.2)
        y_value_for_text = ((np.max(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j]) - np.min(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])) / 2) + np.min(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df_sorted[j])
        plt.text(520,y_value_for_text, s='540', rotation = 90, c= '#1f77b4', alpha = 0.5)

Best classification metrics at 140 neighbors

In [ ]:
best_score_ss = pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'Silhouette Score', ascending = False).iloc[0]['n_neigbors']
best_score_chs = pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'Calinski-Harabasz Score', ascending = False).iloc[0]['n_neigbors']
best_score_dbs = pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by= 'David-Bouldin Score', ascending = True).iloc[0]['n_neigbors']

print(f"N_neigbors that gave the best clustering metrics: \
      \nFor Silhouette Score : {int(best_score_ss)} \
      \nFor Calinski-Harabasz Score : {int(best_score_chs)} \
      \nFor David-Bouldin Score : {int(best_score_dbs)}")

Comparing the Spectral Clustering and PCA- Spectral Clustering:

In [ ]:
spectral_comparisons = pd.concat([pd.DataFrame(spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by = 'Adjusted_rand_score',ascending = False).iloc[0]),
          pd.DataFrame(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df.sort_values(by = 'Adjusted_rand_score',ascending = False).iloc[0])], axis = 1)
spectral_comparisons.columns = ['Spectral Results', 'PCA Spectral Results'] 
spectral_comparisons

PCA Spectral Clustering is giving slightly worse results in terms of the classification and clustering metrics.

## Agglomerative Clustering

Lets see some base scores before hyper parameter tuning

In [ ]:
agglomerative_cluster_model = AgglomerativeClustering(n_clusters = 2)
agglomerative_cluster_model.fit(X_all)
clustering_performance_evaluation
classification_using_clustering_performance_evaluation(y_all, agglomerative_cluster_model.labels_, print_output = True, classification= False)
clustering_performance_evaluation(X_all, agglomerative_cluster_model.labels_, print_output = True)

In [ ]:
#Create a dataframe for storing scores and parameters used.
agglomerative_cluster_hyperparameter_tuning_df = pd.DataFrame({'affinity': [],
                                                             'linkage': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})
for affinity in ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']:
    for linkage in ['ward', 'average', 'complete', 'single']:
        #ward only works with euclidean so pass on all non euclidean affinities
        if (linkage == 'ward') & np.any([affinity == 'l1' or
                                      affinity == 'l2' or
                                      affinity == 'manhattan' or
                                      affinity == 'cosine']):
            pass
        else:
            agglomerative_cluster_model= AgglomerativeClustering(n_clusters = 2,
                                                       affinity = affinity,
                                                       linkage = linkage)
            agglomerative_cluster_model.fit(X_all)
            h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, agglomerative_cluster_model.labels_ , classification= False)
            ss, chs, dbs =  clustering_performance_evaluation(X_all, agglomerative_cluster_model.labels_)
            agglomerative_cluster_hyperparameter_tuning_df = \
                agglomerative_cluster_hyperparameter_tuning_df.append({'affinity': affinity,
                                                                     'linkage': linkage,
                                                                     'Homogeneity Score': h,
                                                                     'Completeness Score': c,
                                                                     'V_measure Score': v,
                                                                     'Accuracy Score': a,
                                                                     'Balanced Accuracy Score': b,
                                                                     'F1 Score': f1,
                                                                     'Adjusted_rand_score': ars,
                                                                     'Silhouette Score': ss,
                                                                     'Calinski-Harabasz Score': chs,
                                                                     'David-Bouldin Score': dbs}, ignore_index = True)                                                             

In [ ]:
agglomerative_cluster_hyperparameter_tuning_df = agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'Adjusted_rand_score', ascending = False)
agglomerative_cluster_hyperparameter_tuning_df

Tuning has produced significantly better scores.

In [ ]:
#combine parameters to one name
agglomerative_cluster_hyperparameter_tuning_df['type'] = agglomerative_cluster_hyperparameter_tuning_df[['affinity','linkage']].apply(' '.join, axis = 1)

In [ ]:
#plot the various scores
fig = plt.figure(figsize = (10,40))
for i,j in zip(range(1,11), ['Homogeneity Score',
       'Completeness Score', 'V_measure Score', 'Accuracy Score',
       'Balanced Accuracy Score', 'F1 Score', 'Adjusted_rand_score',
       'Silhouette Score', 'Calinski-Harabasz Score', 'David-Bouldin Score']):
    fig.add_subplot(5,2,i)
    plt.plot(agglomerative_cluster_hyperparameter_tuning_df['type'],
            agglomerative_cluster_hyperparameter_tuning_df[j])
    plt.title(f"{j}\n Vs Agglomerative Hyperparameters")
    plt.xlabel(f"Hyperparameters")
    plt.xticks(rotation = 90)
    plt.ylabel(f"{j}")

plt.tight_layout()
plt.show()

In [ ]:
print(f"Best Classification Parameters overall is for {agglomerative_cluster_hyperparameter_tuning_df['type'].iloc[0]}")

In [ ]:
best_score_ss = agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'Silhouette Score', ascending = False).iloc[0]['type']
best_score_chs = agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'Calinski-Harabasz Score', ascending = False).iloc[0]['type']
best_score_dbs = agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'David-Bouldin Score', ascending = True).iloc[0]['type']

print(f"Parameters that gave the best clustering metrics: \
      \nFor Silhouette Score : {(best_score_ss)} \
      \nFor Calinski-Harabasz Score : {(best_score_chs)} \
      \nFor David-Bouldin Score : {(best_score_dbs)}")

Next we will compare this with agglomerative clustering using the PCA data

In [ ]:
#Create a dataframe for storing scores and parameters used.
pca_agglomerative_cluster_hyperparameter_tuning_df = pd.DataFrame({'affinity': [],
                                                             'linkage': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})
for affinity in ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']:
    for linkage in ['ward', 'average', 'complete', 'single']:
        #ward only works with euclidean so pass on all non euclidean affinities
        if (linkage == 'ward') & np.any([affinity == 'l1' or
                                      affinity == 'l2' or
                                      affinity == 'manhattan' or
                                      affinity == 'cosine']):
            pass
        else:
            pca_agglomerative_cluster_model= AgglomerativeClustering(n_clusters = 2,
                                                       affinity = affinity,
                                                       linkage = linkage)
            pca_agglomerative_cluster_model.fit(X_pca[:,:6])
            h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, pca_agglomerative_cluster_model.labels_ , classification= False)
            ss, chs, dbs =  clustering_performance_evaluation(X_all, pca_agglomerative_cluster_model.labels_)
            pca_agglomerative_cluster_hyperparameter_tuning_df = \
                pca_agglomerative_cluster_hyperparameter_tuning_df.append({'affinity': affinity,
                                                                     'linkage': linkage,
                                                                     'Homogeneity Score': h,
                                                                     'Completeness Score': c,
                                                                     'V_measure Score': v,
                                                                     'Accuracy Score': a,
                                                                     'Balanced Accuracy Score': b,
                                                                     'F1 Score': f1,
                                                                     'Adjusted_rand_score': ars,
                                                                     'Silhouette Score': ss,
                                                                     'Calinski-Harabasz Score': chs,
                                                                     'David-Bouldin Score': dbs}, ignore_index = True)                                                             

In [ ]:
pca_agglomerative_cluster_hyperparameter_tuning_df = pca_agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'Adjusted_rand_score', ascending = False)
pca_agglomerative_cluster_hyperparameter_tuning_df

In [ ]:
pca_agglomerative_cluster_hyperparameter_tuning_df['type'] = pca_agglomerative_cluster_hyperparameter_tuning_df[['affinity','linkage']].apply(' '.join, axis = 1)

In [ ]:
#plot the various scores
fig = plt.figure(figsize = (10,40))
for i,j in zip(range(1,11), ['Homogeneity Score',
       'Completeness Score', 'V_measure Score', 'Accuracy Score',
       'Balanced Accuracy Score', 'F1 Score', 'Adjusted_rand_score',
       'Silhouette Score', 'Calinski-Harabasz Score', 'David-Bouldin Score']):
    fig.add_subplot(5,2,i)
    plt.plot(pca_agglomerative_cluster_hyperparameter_tuning_df['type'],
            pca_agglomerative_cluster_hyperparameter_tuning_df[j])
    plt.title(f"{j}\n Vs Agglomerative Hyperparameters")
    plt.xlabel(f"Hyperparameters")
    plt.xticks(rotation = 90)
    plt.ylabel(f"{j}")

plt.tight_layout()
plt.show()

In [ ]:
print(f"Best Classification Parameters overall is for {pca_agglomerative_cluster_hyperparameter_tuning_df['type'].iloc[0]}")

In [ ]:
best_score_ss = pca_agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'Silhouette Score', ascending = False).iloc[0]['type']
best_score_chs = pca_agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'Calinski-Harabasz Score', ascending = False).iloc[0]['type']
best_score_dbs = pca_agglomerative_cluster_hyperparameter_tuning_df.sort_values(by= 'David-Bouldin Score', ascending = True).iloc[0]['type']

print(f"Parameters that gave the best clustering metrics: \
      \nFor Silhouette Score : {(best_score_ss)} \
      \nFor Calinski-Harabasz Score : {(best_score_chs)} \
      \nFor David-Bouldin Score : {(best_score_dbs)}")

Comparing the Agglomerative Clustering and PCA- Agglomerative Clustering:

In [ ]:
agglomerative_comparisons = pd.concat([pd.DataFrame(agglomerative_cluster_hyperparameter_tuning_df.sort_values(by = 'Adjusted_rand_score',ascending = False).iloc[0]),
          pd.DataFrame(pca_agglomerative_cluster_hyperparameter_tuning_df.sort_values(by = 'Adjusted_rand_score',ascending = False).iloc[0])], axis = 1)
agglomerative_comparisons.columns = ['Agglomerative Results', 'PCA Agglomerative Results'] 
agglomerative_comparisons

PCA is giving worse scores overall

## Gaussian Mixture Model

GMM assume that all data points are genrated from a mixture of certain number of Gaussian distribution's, and each of these represents a cluster. 

In [ ]:
# Standard Scaler
# Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual features do not more or less look like standard normally distributed data (e.g. Gaussian with 0 mean and unit variance)
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
#X_scaled = StandardScaler().fit_transform(X)

### Modeling with Scaled data (Without PCA)

In [ ]:
## Gausian Mixture
gm = GaussianMixture(n_components=2, covariance_type="full")
gm.fit(X_all)

In [ ]:
print(f"Weights are: {gm.weights_}")
print(f"Converged status is: {gm.converged_}")
print(f"Number of iterations needed for converging: {gm.n_iter_}")
#print(f"Means with and without PCA and Scaling are {gm.means_} and {gm_scaled_pca.means_} respectively")
#print(f"Covariances with and without PCA and Scaling are {gm.covariances_} and {gm_scaled_pca.covariances_} respectively")

In [ ]:
gm_pred = gm.predict(X_all)

In [ ]:
gm.predict_proba(X_all)

In [ ]:
# b) Evaluate the clustering methods using appropriate metrics such as the Adjusted Rand index, Homogeneity, Completeness and V-Measure, 
#    using the ground truth.
# Evaluation Techniques --> https://scikit-learn.org/stable/modules/model_evaluation.html#the-scoring-parameter-defining-model-evaluation-rules
# Adjusted Rand index, Homogeneity, Completeness and V-Measure, using the ground truth.

clustering_performance_evaluation(X_all, gm_pred, print_output = True)

print("\n\n")
classification_using_clustering_performance_evaluation(y_all, gm_pred, print_output = True, classification= False)

#### Hyper Parameter tuning with Scaled data (Without PCA)

In [ ]:
#Create a dataframe for storing scores and parameters for Gaussian Mixture Model (https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)
gmm_clustering_hyperparameter_tuning_df = pd.DataFrame({'covariance_type': [],
                                                             'init_params': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

In [ ]:
for covariance_type in ['full','tied','diag', 'spherical']:
  for param in ['kmeans','random']:
    gmm_model = GaussianMixture(n_components=2, 
                                covariance_type=covariance_type,
                                init_params=param
                                )
    gmm_model.fit(X_all)
    gmm_pred = gmm_model.predict(X_all)
    h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, gmm_pred, classification= True)
    
    ss, chs, dbs =  clustering_performance_evaluation(X_all, gmm_pred)

    gmm_clustering_hyperparameter_tuning_df = \
                gmm_clustering_hyperparameter_tuning_df.append({'covariance_type': covariance_type,
                                                                     'init_params': param,
                                                                     'Homogeneity Score': h,
                                                                     'Completeness Score': c,
                                                                     'V_measure Score': v,
                                                                     'Accuracy Score': a,
                                                                     'Balanced Accuracy Score': b,
                                                                     'F1 Score': f1,
                                                                     'Adjusted_rand_score': ars,
                                                                     'Silhouette Score': ss,
                                                                     'Calinski-Harabasz Score': chs,
                                                                     'David-Bouldin Score': dbs}, ignore_index = True)  


In [ ]:
gmm_clustering_hyperparameter_tuning_df = gmm_clustering_hyperparameter_tuning_df.sort_values(by= 'Adjusted_rand_score', ascending = False)
gmm_clustering_hyperparameter_tuning_df

### Modeling with PCA data

In [ ]:
gm_scaled_pca = GaussianMixture(n_components=2, covariance_type="full")
gm_scaled_pca.fit(X_pca)
gm_pred_scaled_pca = gm_scaled_pca.predict(X_pca)

In [ ]:
gm_scaled_pca.predict_proba(X_pca)

In [ ]:
print(f"Weights are: {gm_scaled_pca.weights_}")
print(f"Converged status is: {gm_scaled_pca.converged_}")
print(f"Number of iterations needed for converging: {gm_scaled_pca.n_iter_}")

In [ ]:
print("Converged: ",gm_scaled_pca.converged_)
print("Number of Iterations: ",gm_scaled_pca.n_iter_)
print("Number of Features: ",gm_scaled_pca.n_features_in_)
print(f"Means with shape: {gm_scaled_pca.means_.shape} and values: {gm_scaled_pca.means_}")
print(f"Weight with shape: {gm_scaled_pca.weights_.shape} and values: {gm_scaled_pca.weights_}")

In [ ]:
print("\nScores on scaled and PCA data")
clustering_performance_evaluation(X_pca, gm_pred_scaled_pca, print_output = True)

print("\nScores on scaled and PCA data")
classification_using_clustering_performance_evaluation(y_all, gm_pred_scaled_pca, print_output = True, classification= False)

#### Hyper Parameter tuning with PCA *data*

In [ ]:
#Create a dataframe for storing scores and parameters for Gaussian Mixture Model (https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)
gmm_clustering_hyperparameter_tuning_pca_df = pd.DataFrame({'covariance_type': [],
                                                             'init_params': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

In [ ]:
for covariance_type in ['full','tied','diag', 'spherical']:
  for param in ['kmeans','random']:
    gmm_model = GaussianMixture(n_components=2, 
                                covariance_type=covariance_type,
                                init_params=param
                                )
    gmm_model.fit(X_pca)
    gmm_pred = gmm_model.predict(X_pca)

    h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, gmm_pred , classification= True)
    
    ss, chs, dbs =  clustering_performance_evaluation(X_pca, gmm_pred)

    gmm_clustering_hyperparameter_tuning_pca_df = \
                gmm_clustering_hyperparameter_tuning_pca_df.append({'covariance_type': covariance_type,
                                                                     'init_params': param,
                                                                     'Homogeneity Score': h,
                                                                     'Completeness Score': c,
                                                                     'V_measure Score': v,
                                                                     'Accuracy Score': a,
                                                                     'Balanced Accuracy Score': b,
                                                                     'F1 Score': f1,
                                                                     'Adjusted_rand_score': ars,
                                                                     'Silhouette Score': ss,
                                                                     'Calinski-Harabasz Score': chs,
                                                                     'David-Bouldin Score': dbs}, ignore_index = True)  


In [ ]:
gmm_clustering_hyperparameter_tuning_pca_df = gmm_clustering_hyperparameter_tuning_pca_df.sort_values(by= 'Adjusted_rand_score', ascending = False)
gmm_clustering_hyperparameter_tuning_pca_df

In [ ]:
# Scatter plots
fig, ((ax1, ax2, ax3)) = plt.subplots(1, 3, sharey=True, tight_layout=True, figsize=(20,10))
fig.suptitle("Cluster Distribution", fontsize=30, fontweight='bold', y=1.05) # y value to place sup title slightly outside the figure.

ax1.scatter(X_all[:,0][y_all == 0], X_all[:,1][y_all == 0], label = 'Benign', alpha=0.45)
ax1.scatter(X_all[:,0][y_all == 1], X_all[:,1][y_all == 1], label = 'Malignant', alpha=0.45)
ax1.set_title("Actual clusters", fontsize=15, fontweight='bold')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlabel(X_columns[0])
ax1.set_ylabel(X_columns[1])
ax1.legend()

ax2.scatter(X_all[:,0][gm_pred == 1], X_all[:,1][gm_pred == 1], label = 'Benign', alpha=0.45)
ax2.scatter(X_all[:,0][gm_pred == 0], X_all[:,1][gm_pred == 0], label = 'Malignant', alpha=0.45)
ax2.set_title("Modeled Clusters on Scaled Data", fontsize=15, fontweight='bold')
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(X_columns[0])
ax2.legend()

ax3.scatter(X_pca[:,0][gm_pred_scaled_pca == 0], X_pca[:,1][gm_pred_scaled_pca == 0], label = 'Benign', alpha=0.45)
ax3.scatter(X_pca[:,0][gm_pred_scaled_pca == 1], X_pca[:,1][gm_pred_scaled_pca == 1], label = 'Malignant', alpha=0.45)
ax3.set_title("Modeled Clusters on PCA data", fontsize=15, fontweight='bold')
ax3.set_xticks([])
ax3.set_yticks([])
ax3.set_xlabel(X_columns[0])
ax3.legend()

plt.tight_layout()
fig.show()

## Mean Shift

Mean shift clustering is a hierarchial clustering algorithm which aims to discover blobs in a smooth density of samples.It works by updating candidate for centroids to be the mean of the point within a certain region.


In [ ]:
bandwidth = estimate_bandwidth(X_all, quantile=0.2, n_samples=500)
bandwidth

### Modeling with Scaled data

In [ ]:
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X_all)
ms_pred = ms.predict(X_all)

In [ ]:
np.unique(ms_pred)

### Modeling with PCA data

In [ ]:
ms_scaled_pca = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms_scaled_pca.fit(X_pca)
ms_pred_scaled_pca = ms_scaled_pca.predict(X_pca)

In [ ]:
np.unique(ms_pred_scaled_pca)

In [ ]:
# Scatter plots
fig, ((ax1, ax2, ax3)) = plt.subplots(1, 3, sharey=True, tight_layout=True, figsize=(20,10))
fig.suptitle("Cluster Distribution", fontsize=30, fontweight='bold', y=1.05) # y value to place sup title slightly outside the figure.

ax1.scatter(X_all[:,0][y_all == 0], X_all[:,1][y_all == 0], label = 'Benign', alpha=0.45)
ax1.scatter(X_all[:,0][y_all == 1], X_all[:,1][y_all == 1], label = 'Malignant', alpha=0.45)
ax1.set_title("Actual clusters", fontsize=15, fontweight='bold')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlabel(X_columns[0])
ax1.set_ylabel(X_columns[1])
ax1.legend()

ax2.scatter(X_all[:,0][ms_pred == 1], X_all[:,1][ms_pred == 1], label = 'Benign', alpha=0.45)
ax2.scatter(X_all[:,0][ms_pred == 0], X_all[:,1][ms_pred == 0], label = 'Malignant', alpha=0.45)
ax2.set_title("Modeled Clusters on Scaled Data", fontsize=15, fontweight='bold')
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(X_columns[0])
ax2.legend()

ax3.scatter(X_pca[:,0][ms_pred_scaled_pca == 1], X_pca[:,1][ms_pred_scaled_pca == 1], label = 'Benign', alpha=0.45)
ax3.scatter(X_pca[:,0][ms_pred_scaled_pca == 0], X_pca[:,1][ms_pred_scaled_pca == 0], label = 'Malignant', alpha=0.45)
ax3.set_title("Modeled Clusters on PCA data", fontsize=15, fontweight='bold')
ax3.set_xticks([])
ax3.set_yticks([])
ax3.set_xlabel(X_columns[0])
ax3.legend()

plt.tight_layout()
fig.show()

In [ ]:
print("\nScores on scaled and PCA data")
clustering_performance_evaluation(X_pca, ms_pred_scaled_pca, print_output = True)

In [ ]:
print("Scores on not scaled data")
classification_using_clustering_performance_evaluation(y_all, ms_pred, print_output = True, classification= False)

In [ ]:
#Create a dataframe for storing scores and parameters for Gaussian Mixture Model (https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)
ms_clustering_hyperparameter_tuning_pca_df = pd.DataFrame({'covariance_type': [],
                                                             'cluster_all': [],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

On Upscaled data - no clustering
ON PCA - 5 labels predicted. No way to spcify number of clusters. Bad choice for Predicting in this data, hence no hyper parameter tuning

## K-Means Clustering

Kmaens clustering is an iterative algorithm that tries to seprate the dataset in to *K* pre-define distinct non overlappong clusters. It tries to make the intracluster data points as similar as possible while also keeping the clusters as fas as possible.

https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a

In [ ]:
#Standard scalar to perform scaling of features
#X = StandardScaler().fit_transform(X_all)

#K-Means Unsupervisied Machine Learning
kmeans = KMeans(n_clusters = 2, init='k-means++', max_iter=300, random_state = None, n_init = 10)
kmeans_model = kmeans.fit(X_all)
kmeans_model_predict = kmeans.predict(X_all)
kmeans_predictions = kmeans.labels_
kmeans_predictions
print(kmeans.get_params)

In [ ]:
kmeans_predictions

In [ ]:
#Evaluating model performance on clustering metrics
clustering_performance_evaluation(X_all, kmeans_model.labels_, print_output = True)

In [ ]:
#Evaluating model performance on classification metrics 
value = classification_using_clustering_performance_evaluation(y_all, kmeans_model.labels_, print_output= True, classification= False)

In [ ]:
#Hyperparameter tuning for K-Means:

#Creating Dataframe to store score

kmeans_cluster_hyperparameter_tuning_df = pd.DataFrame({'n_init': [],
                                                             'init' :[],
                                                             'algorithm' :[],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

In [ ]:
for n_init in [10,100,1000]:
  for init in ['k-means++', 'random']:
    for algorithm in ['auto', 'full','elkan']:
      kmeans_cluster_model = KMeans(n_clusters=2,
                                    max_iter=300,
                                    random_state= 1,
                                    n_init = n_init,
                                    init = init,
                                    algorithm = algorithm)
      kmeans_cluster_model.fit(X_all)
      h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, kmeans_cluster_model.labels_, classification = False)
      ss, chs,dbs = clustering_performance_evaluation(X_all, kmeans_cluster_model.labels_)
      kmeans_cluster_hyperparameter_tuning_df = \
        kmeans_cluster_hyperparameter_tuning_df.append({'n_init': n_init,
                                                        'init' : init,
                                                        'algorithm' : algorithm,
                                                        'Homogeneity Score': h,
                                                        'Completeness Score': c,
                                                        'V_measure Score': v,
                                                        'Accuracy Score': a,
                                                        'Balanced Accuracy Score': b,
                                                        'F1 Score': f1,
                                                        'Adjusted_rand_score': ars,
                                                        'Silhouette Score': ss,
                                                        'Calinski-Harabasz Score': chs,
                                                        'David-Bouldin Score': dbs}, ignore_index = True)

In [ ]:
#Printing scores and parameters in descending order

kmeans_cluster_hyperparameter_tuning_df.sort_values(by = 'Adjusted_rand_score', ascending = False)

In [ ]:
#Using Principal Component Analysis (Linear dimensionality reduction)

#pca = PCA(n_components=2)
#pca.fit(X)

#x_pca = pca.transform(X)

kmeans_pca = KMeans(n_clusters = 2, init='k-means++', max_iter=300, random_state = 1, n_init = 10)
kmeans_pca_model = kmeans.fit(X_pca)
kmeans_pca_model_predict = kmeans.predict(X_pca)
kmeans_pca_predictions = kmeans.labels_
kmeans_pca_predictions
print(kmeans_pca.get_params)



In [ ]:
#Evaluating model performance on clustering metrics
clustering_performance_evaluation(X_pca, kmeans_pca_model.labels_, print_output = True)

In [ ]:

#Evaluating model performance on classification metrics
value = classification_using_clustering_performance_evaluation(y_all, kmeans_pca_model.labels_, print_output= True, classification= False)

In [ ]:
#Hyperparameter tuning for PCA K-Means:

#Creating Dataframe to store score

kmeans_pca_cluster_hyperparameter_tuning_df = pd.DataFrame({'n_init': [],
                                                             'init' :[],
                                                             'algorithm' :[],
                                                             'Homogeneity Score': [],
                                                             'Completeness Score': [],
                                                             'V_measure Score': [],
                                                             'Accuracy Score': [],
                                                             'Balanced Accuracy Score': [],
                                                             'F1 Score': [],
                                                             'Adjusted_rand_score': [],
                                                             'Silhouette Score': [],
                                                             'Calinski-Harabasz Score': [],
                                                             'David-Bouldin Score': []})

In [ ]:
for n_init in [10,100,1000]:
  for init in ['k-means++', 'random']:
    for algorithm in ['auto', 'full','elkan']:
      kmeans_pca_cluster_model = KMeans(n_clusters=2,
                                    max_iter=300,
                                    random_state= 1,
                                    n_init = n_init,
                                    init = init,
                                    algorithm = algorithm)
      kmeans_pca_cluster_model.fit(X_pca)
      h,c,v,a,b,f1,ars = classification_using_clustering_performance_evaluation(y_all, kmeans_pca_cluster_model.labels_, classification = False)
      ss, chs,dbs = clustering_performance_evaluation(X_pca, kmeans_pca_cluster_model.labels_)
      kmeans_pca_cluster_hyperparameter_tuning_df = \
        kmeans_pca_cluster_hyperparameter_tuning_df.append({'n_init': n_init,
                                                        'init' : init,
                                                        'algorithm' : algorithm,
                                                        'Homogeneity Score': h,
                                                        'Completeness Score': c,
                                                        'V_measure Score': v,
                                                        'Accuracy Score': a,
                                                        'Balanced Accuracy Score': b,
                                                        'F1 Score': f1,
                                                        'Adjusted_rand_score': ars,
                                                        'Silhouette Score': ss,
                                                        'Calinski-Harabasz Score': chs,
                                                        'David-Bouldin Score': dbs}, ignore_index = True)

In [ ]:
kmeans_pca_cluster_hyperparameter_tuning_df.sort_values(by = 'Adjusted_rand_score', ascending = False)

In [ ]:
#kernel PCA

kpca = KernelPCA(n_components=2, kernel='linear')
#kpca = KernelPCA(n_components=2, kernel='cosine')
#kpca = KernelPCA(n_components=2, kernel='rbf')
kpca.fit(X_all)

X_kpca = kpca.transform(X_all)

kmeans_kpca = KMeans(n_clusters = 2, init='k-means++', max_iter=300, random_state = None, n_init = 10)
kmeans_kpca_model = kmeans.fit(X_kpca)
kmeans_kpca_model_predict = kmeans.predict(X_kpca)
kmeans_kpca_predictions = kmeans.labels_
kmeans_kpca_predictions
print(kmeans_kpca.get_params)



In [ ]:
clustering_performance_evaluation(X_kpca, kmeans_kpca_model.labels_, print_output = True)

In [ ]:
# Scatter plots
fig, ((ax1, ax2, ax3, ax4)) = plt.subplots(1, 4, sharey=True, tight_layout=True, figsize=(20,10))
fig.suptitle("Cluster Distribution", fontsize=30, fontweight='bold', y=1.05) # y value to place sup title slightly outside the figure.

ax1.scatter(X_all[:,0][y_all == 0], X_all[:,1][y_all == 0], label = 'Benign', alpha=0.45)
ax1.scatter(X_all[:,0][y_all == 1], X_all[:,1][y_all == 1], label = 'Malignant', alpha=0.45)
ax1.set_title("Actual clusters", fontsize=15, fontweight='bold')
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlabel(X_columns[0])
ax1.set_ylabel(X_columns[1])
ax1.legend()

ax2.scatter(X_all[:,0][kmeans_model_predict == 1], X_all[:,1][kmeans_model_predict == 1], label = 'Benign', alpha=0.45)
ax2.scatter(X_all[:,0][kmeans_model_predict == 0], X_all[:,1][kmeans_model_predict == 0], label = 'Malignant', alpha=0.45)
ax2.set_title("Modeled Clusters on Scaled Data", fontsize=15, fontweight='bold')
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(X_columns[0])
ax2.legend()

ax3.scatter(X_pca[:,0][kmeans_pca_model_predict == 1], X_pca[:,1][kmeans_pca_model_predict == 1], label = 'Benign', alpha=0.45)
ax3.scatter(X_pca[:,0][kmeans_pca_model_predict == 0], X_pca[:,1][kmeans_pca_model_predict == 0], label = 'Malignant', alpha=0.45)
ax3.set_title("Modeled Clusters on PCA data", fontsize=15, fontweight='bold')
ax3.set_xticks([])
ax3.set_yticks([])
ax3.set_xlabel(X_columns[0])
ax3.legend()

ax4.scatter(X_kpca[:,0][kmeans_kpca_model_predict == 1], X_kpca[:,1][kmeans_kpca_model_predict == 1], label = 'Benign', alpha=0.45)
ax4.scatter(X_kpca[:,0][kmeans_kpca_model_predict == 0], X_kpca[:,1][kmeans_kpca_model_predict == 0], label = 'Malignant', alpha=0.45)
ax4.set_title("Modeled Clusters on K-PCA data", fontsize=15, fontweight='bold')
ax4.set_xticks([])
ax4.set_yticks([])
ax4.set_xlabel(X_columns[0])
ax4.legend()

plt.tight_layout()
fig.show()

## DBSCAN Clustering

DBSCAN is a clustering algorith that doesn't require the number of clusters to be predetermined as a hyperparameter. However, it does require epsilon and min_samples to be tuned for better clustering.

The algorithm was implemented on the the PCA reduced dataset to determine if it provided a good clustering result.

Strategy for  DBSCAN Clustering are as follows:
1. Determine initial hyperparameters values for eps and min_samples.
2. Fine Tune hyperparameters around identified values in step 1 for best clustering.
3. Evaluate Clustering Performance



In [ ]:
#Storing features in X dataframe and target label (y) as numpy array
X_db = X.to_numpy().copy()
y_db_old = y.to_numpy().copy()
y_db = y_db_old.reshape((-1,1))

#Concatenating both X_db And Y_db arrays for plotting purposes later
x_db_concat = np.concatenate((X_db,y_db), axis=1)
benign = x_db_concat[np.where(x_db_concat[:,-1]==1)]
malignant = x_db_concat[np.where(x_db_concat[:,-1]==0)]

#PCA reduction
db_pca = PCA()
db_x_red =db_pca.fit_transform(X_db)

db_pca_var_perc = [100*i for i in db_pca.explained_variance_ratio_]

Earlier, it was shown that the first 7 PCA components cumulatively explained 85% of the variance in the Data. With this information, PCA was conducted once again to extract only the first 7 PCA Components for clustering.

In [ ]:
type(X_db)

In [ ]:
#plotting the explained variance ratio as percentage

fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
ax.bar(x=[i for i in range(len(db_pca_var_perc))],height=db_pca_var_perc)
ax.set_title('PCA components explanied variance in percentage')
ax.set_xlabel('Principal Component Axis')
ax.set_ylabel('Percentage of explained variance')

db_pca_perc_sum = 0
db_pca_count = 0

for i in db_pca_var_perc:
  if(db_pca_perc_sum < 85):
    db_pca_perc_sum += i
    db_pca_count += 1

print(f'{db_pca_count} PCA components produces a cumulative variance of {round(db_pca_perc_sum,2)}% and can be used for further study')

In [ ]:
#PCA To produce 7 components
db_x_red = PCA(n_components=7).fit_transform(X_db)
x_red_concat = np.concatenate((db_x_red,y_db), axis=1)

benign = x_red_concat[np.where(x_red_concat[:,-1]==1)]
malignant = x_red_concat[np.where(x_red_concat[:,-1]==0)]

bening_1_pca_1 = benign[:,0]
bening_1_pca_2 = benign[:,1]

malignant_1_pca_1 = malignant[:,0]
malignant_1_pca_2 = malignant[:,1]


fig,ax = plt.subplots(nrows=1,ncols=1)
ax.scatter(bening_1_pca_1,bening_1_pca_2,label = 'benign',alpha =0.1)
ax.scatter(malignant_1_pca_1,malignant_1_pca_2,label = 'malignant',alpha=0.1)
ax.set_xlabel('Principal component 1')
ax.set_ylabel('Principal component 2')
ax.legend()

In [ ]:
'''General practise suggested is to use 2 times the dimension.Since we're using 7 PCA 
components we ill be using a value of 14  (Schubert et al., 2017)'''
min_samples = 14


'''As per Rahmah and Sitanggang, 2016 a solution to identify a good eps value 
is to run a nearest neighbour algorithm with the nearest neighbours being the 
same value as the min_samples chosen. Average distance between each sample and 
its identified neighbour is calculated, sorted and plotted. The elbow point in the
plotted grap (point with the biggest curve) is chosen as the eps value'''

knearest = NearestNeighbors(n_neighbors=min_samples).fit(db_x_red)
cal_dist,ind = knearest.kneighbors(db_x_red)
sort_dist = np.sort(cal_dist,axis=0)[:,1]

fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(8,8))
ax.plot(sort_dist)
ax.set_xlabel('Data Point Index')
ax.set_ylabel('Average Distance')
ax.set_title('Curve to identify optimum eps value for DBSCAN')
fig.show()

The biggest curvature appears to fall within the range of 2 to 6. Parameter tuning will now be conducted in this range for the selected min_sample value of 12.

In [ ]:
#choose what evaluation metric to use
#create df
#hyper parameter tuning
#graph showing how parameter changes affect scores
  

#Evaluation
parameter_score_dict = {'eps':[],'min_samples':[],'cluster_count':[],'homogeneity_score': [],
'completeness_score':[],'v_measure_score': [],'adjusted_rand_score': [],	
'silhouette_score': [],'calinski-harabasz_score': [],	'david-bouldin_score': []}

for i in np.arange(2,6,0.1):
  dbscan = cluster.DBSCAN(eps=i, min_samples=min_samples)
  clust_labels = dbscan.fit_predict(db_x_red)
  noise_count = list(clust_labels).count(-1)
  
  if noise_count > 0:
    clust_count =  len(set(clust_labels)) - 1
  else:
    clust_count =  len(set(clust_labels))
  
  parameter_score_dict['eps'].append(i)
  parameter_score_dict['min_samples'].append(min_samples)
  parameter_score_dict['cluster_count'].append(clust_count)
  parameter_score_dict['homogeneity_score'].append(homogeneity_score(y_db_old,clust_labels))
  parameter_score_dict['completeness_score'].append(completeness_score(y_db_old,clust_labels))
  parameter_score_dict['v_measure_score'].append(v_measure_score(y_db_old,clust_labels))
  parameter_score_dict['adjusted_rand_score'].append(adjusted_rand_score(y_db_old,clust_labels))
  parameter_score_dict['silhouette_score'].append(silhouette_score(db_x_red,clust_labels))
  parameter_score_dict['calinski-harabasz_score'].append(calinski_harabasz_score(db_x_red,clust_labels))
  parameter_score_dict['david-bouldin_score'].append(davies_bouldin_score(db_x_red,clust_labels))

results = np.concatenate((np.array(parameter_score_dict['eps']).reshape((-1,1)),np.array(parameter_score_dict['min_samples']).reshape((-1,1)),
                     np.array(parameter_score_dict['cluster_count']).reshape((-1,1)),
                     np.array(parameter_score_dict['homogeneity_score']).reshape((-1,1)),
                     np.array(parameter_score_dict['completeness_score']).reshape((-1,1)),
                     np.array(parameter_score_dict['v_measure_score']).reshape((-1,1)),np.array(parameter_score_dict['adjusted_rand_score']).reshape((-1,1)),
                     np.array(parameter_score_dict['silhouette_score']).reshape((-1,1)),np.array(parameter_score_dict['calinski-harabasz_score']).reshape((-1,1)),
                     np.array(parameter_score_dict['david-bouldin_score']).reshape((-1,1))),axis=1)

In [ ]:
#results in numpy format

'''
Column information is as follows
column 1 - eps
column 2 - min_samples
column 3 - cluster_count
column 4 - homogeneity_score
column 5 - completeness_score
column 6 - v_measure_score
column 7 - adjusted_rand_score
column 8 - silhouette_score
column 9 - calinski-harabasz_score
column 10 - david-bouldin_score
'''
print(results)

In [ ]:
#The same information presented using pandas for easy visualisation
results_df =pd.DataFrame(parameter_score_dict)
res_df_cols = results_df.columns
results_df

Since the purpose is to identify two clusters as per our target values the numpy array is filtered to identify results thats generated 2 clusters.

In [ ]:
#filtering for two clusters
two_cluster_results = results[results[:,2]==2]
two_cluster_results
filtered_results = results[np.where(results[:,2]==2)]

In [ ]:
dbscan_results = pd.DataFrame(filtered_results,columns=res_df_cols)
dbscan_results

In [ ]:
eps_arr = two_cluster_results[:,0]
min_samples_arr = two_cluster_results[:,1]
two_cluster_results[:,1]

best_sil_score = max(two_cluster_results[:,7])
eps_sil_score = two_cluster_results[np.where(two_cluster_results[:,7]==best_sil_score)][0][0]
best_ch_score = max(two_cluster_results[:,8])
eps_ch_score = two_cluster_results[np.where(two_cluster_results[:,8]==best_ch_score)][0][0]
best_db_score = max(two_cluster_results[:,9])
eps_db_score = two_cluster_results[np.where(two_cluster_results[:,9]==best_db_score)][0][0]
best_rand_score =  max(two_cluster_results[:,6])
eps_rand_score = two_cluster_results[np.where(two_cluster_results[:,6]==best_rand_score)][0][0]

In [ ]:
print(f'The best parameters for the best identified silhouette score {round(best_sil_score,2)} are:  eps - {round(eps_sil_score,1)}  min_samples -{min_samples}')
print(f'The best parameters for the best identified calinski-harabasz score {round(best_ch_score,2)} are:  eps - {round(eps_ch_score,1)}  min_samples -{min_samples}')
print(f'The best parameters for the best identified david-bouldin score {round(best_db_score,2)} are:  eps - {round(eps_db_score,1)}  min_samples -{min_samples}')
print(f'The best parameters for the best identified Adjusted Rand score {round(best_rand_score,2)} are:  eps - {round(eps_rand_score,1)}  min_samples -{min_samples}')

In [ ]:
dbscan = cluster.DBSCAN(eps=eps_sil_score, min_samples=min_samples)
clust_labels = dbscan.fit_predict(db_x_red)

new_concat = np.concatenate((db_x_red,np.array(clust_labels).reshape((-1,1))),axis=1)

out_x1 = new_concat[new_concat[:,-1]==-1][:,0]
out_x2 = new_concat[new_concat[:,-1]==-1][:,1]

clust_1_x1  = new_concat[new_concat[:,-1]==0][:,0]
clust_1_x2  = new_concat[new_concat[:,-1]==0][:,1]

clust_2_x1  = new_concat[new_concat[:,-1]==1][:,0]
clust_2_x2  = new_concat[new_concat[:,-1]==1][:,1]

In [ ]:
#Cluster Visualisation for the best silhouette score
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
ax[1].scatter(bening_1_pca_1,bening_1_pca_2,label = 'benign',alpha =0.1)
ax[1].scatter(malignant_1_pca_1,malignant_1_pca_2,label = 'malignant',alpha=0.1)
ax[1].set_xlabel('Principal component 1')
ax[1].set_ylabel('Principal component 2')
ax[1].legend()
ax[0].set_xlabel('Principal component 1')
ax[0].set_ylabel('Principal component 2')
ax[0].scatter(out_x1,out_x2,alpha =0.5,label = 'outliers')
ax[0].scatter(clust_1_x1,clust_1_x2,alpha =0.5,label = 'cluster 1')
ax[0].scatter(clust_2_x1,clust_2_x2,alpha =0.5,label = 'cluster 2')
ax[0].legend()

In [ ]:
#Cluster Visualisation for the best calinski-harabasz score
dbscan = cluster.DBSCAN(eps=eps_ch_score, min_samples=min_samples)
clust_labels = dbscan.fit_predict(db_x_red)

new_concat = np.concatenate((db_x_red,np.array(clust_labels).reshape((-1,1))),axis=1)

out_x1 = new_concat[new_concat[:,-1]==-1][:,0]
out_x2 = new_concat[new_concat[:,-1]==-1][:,1]

clust_1_x1  = new_concat[new_concat[:,-1]==0][:,0]
clust_1_x2  = new_concat[new_concat[:,-1]==0][:,1]

clust_2_x1  = new_concat[new_concat[:,-1]==1][:,0]
clust_2_x2  = new_concat[new_concat[:,-1]==1][:,1]

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
ax[1].scatter(bening_1_pca_1,bening_1_pca_2,label = 'benign',alpha =0.1)
ax[1].scatter(malignant_1_pca_1,malignant_1_pca_2,label = 'malignant',alpha=0.1)
ax[1].set_xlabel('Principal component 1')
ax[1].set_ylabel('Principal component 2')
ax[1].legend()
ax[0].set_xlabel('Principal component 1')
ax[0].set_ylabel('Principal component 2')
ax[0].scatter(out_x1,out_x2,alpha =0.5,label = 'outliers')
ax[0].scatter(clust_1_x1,clust_1_x2,alpha =0.5,label = 'cluster 1')
ax[0].scatter(clust_2_x1,clust_2_x2,alpha =0.5,label = 'cluster 2')
ax[0].legend()


In [ ]:
#Cluster Visualisation for the best david-bouldin_score
dbscan = cluster.DBSCAN(eps=eps_db_score, min_samples=min_samples)
clust_labels = dbscan.fit_predict(db_x_red)

new_concat = np.concatenate((db_x_red,np.array(clust_labels).reshape((-1,1))),axis=1)

out_x1 = new_concat[new_concat[:,-1]==-1][:,0]
out_x2 = new_concat[new_concat[:,-1]==-1][:,1]

clust_1_x1  = new_concat[new_concat[:,-1]==0][:,0]
clust_1_x2  = new_concat[new_concat[:,-1]==0][:,1]

clust_2_x1  = new_concat[new_concat[:,-1]==1][:,0]
clust_2_x2  = new_concat[new_concat[:,-1]==1][:,1]

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
ax[1].scatter(bening_1_pca_1,bening_1_pca_2,label = 'benign',alpha =0.1)
ax[1].scatter(malignant_1_pca_1,malignant_1_pca_2,label = 'malignant',alpha=0.1)
ax[1].set_xlabel('Principal component 1')
ax[1].set_ylabel('Principal component 2')
ax[1].legend()
ax[0].set_xlabel('Principal component 1')
ax[0].set_ylabel('Principal component 2')
ax[0].scatter(out_x1,out_x2,alpha =0.5,label = 'outliers')
ax[0].scatter(clust_1_x1,clust_1_x2,alpha =0.5,label = 'cluster 1')
ax[0].scatter(clust_2_x1,clust_2_x2,alpha =0.5,label = 'cluster 2')
ax[0].legend()

In [ ]:
#Cluster Visualisation for the best Adjusted Rand score
dbscan = cluster.DBSCAN(eps=eps_rand_score, min_samples=min_samples)
clust_labels = dbscan.fit_predict(db_x_red)

new_concat = np.concatenate((db_x_red,np.array(clust_labels).reshape((-1,1))),axis=1)

out_x1 = new_concat[new_concat[:,-1]==-1][:,0]
out_x2 = new_concat[new_concat[:,-1]==-1][:,1]

clust_1_x1  = new_concat[new_concat[:,-1]==0][:,0]
clust_1_x2  = new_concat[new_concat[:,-1]==0][:,1]

clust_2_x1  = new_concat[new_concat[:,-1]==1][:,0]
clust_2_x2  = new_concat[new_concat[:,-1]==1][:,1]

In [ ]:
#Cluster Visualisation for the best Adjusted Rand score
fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(10,10))
ax[1].scatter(bening_1_pca_1,bening_1_pca_2,label = 'benign',alpha =0.1)
ax[1].scatter(malignant_1_pca_1,malignant_1_pca_2,label = 'malignant',alpha=0.1)
ax[1].set_xlabel('Principal component 1')
ax[1].set_ylabel('Principal component 2')
ax[1].legend()
ax[0].set_xlabel('Principal component 1')
ax[0].set_ylabel('Principal component 2')
ax[0].scatter(out_x1,out_x2,alpha =0.5,label = 'outliers')
ax[0].scatter(clust_1_x1,clust_1_x2,alpha =0.5,label = 'cluster 1')
ax[0].scatter(clust_2_x1,clust_2_x2,alpha =0.5,label = 'cluster 2')
ax[0].legend()

In [ ]:
#DBSCAN on original dataset with no dimensionality reduction

In [ ]:
non_pca_min_samples = 136

non_pca_knearest = NearestNeighbors(n_neighbors=min_samples).fit(X_db)
cal_dist,ind = non_pca_knearest.kneighbors(X_db)
sort_dist = np.sort(cal_dist,axis=0)[:,1]

fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(8,8))
ax.plot(sort_dist)
ax.set_xlabel('Data Point Index')
ax.set_ylabel('Average Distance')
ax.set_title('Curve to identify optimum eps value for DBSCAN')
fig.show()

The biggest curvature appears to fall within the range of 3.5 to 10. Parameter tuning will now be conducted in this range for the selected min_sample value of 136.

For the Original dataset without PCA, the DBSCAN algorithm constantly created a single cluster or no cluster at all. Hence, this approach was abandoned.

In [ ]:
non_pca_parameter_score_dict = {'eps':[],'min_samples':[],'cluster_count':[],'homogeneity_score': [],
'completeness_score':[],'v_measure_score': [],'adjusted_rand_score': [],	
'silhouette_score': [],'calinski-harabasz_score': [],	'david-bouldin_score': []}

for i in np.arange(3.5,10,0.1):
  dbscan = cluster.DBSCAN(eps=i, min_samples=non_pca_min_samples)
  clust_labels = dbscan.fit_predict(X_db)
  noise_count = list(clust_labels).count(-1)
  
  if noise_count > 0:
    clust_count =  len(set(clust_labels)) - 1
  else:
    clust_count =  len(set(clust_labels))
  non_pca_parameter_score_dict['cluster_count'].append(clust_count)


non_pca_parameter_score_dict['cluster_count']

## Comparison of Clustering Methods

In [ ]:
def frame_val_extract(frame,score_name):
  return frame[frame[score_name]==max(frame[score_name])]

spectral_clustering_sil_best = frame_val_extract(spectral_clustering_hyperparameter_tuning_n_neighbours_df,'Adjusted_rand_score')
spectral_clustering_pca_sil_best = frame_val_extract(pca_spectral_clustering_hyperparameter_tuning_n_neighbours_df,'Adjusted_rand_score')
agglomerative_sil_best = frame_val_extract(agglomerative_cluster_hyperparameter_tuning_df,'Adjusted_rand_score')
agglomerative_pca_sil_best = frame_val_extract(pca_agglomerative_cluster_hyperparameter_tuning_df,'Adjusted_rand_score')
gmm_sil_best = frame_val_extract(gmm_clustering_hyperparameter_tuning_df,'Adjusted_rand_score')
gmm_pca_best_sil  = frame_val_extract(gmm_clustering_hyperparameter_tuning_pca_df,'Adjusted_rand_score')
kmean_best_sil  = frame_val_extract(kmeans_cluster_hyperparameter_tuning_df,'Adjusted_rand_score')
kmeans_pca_best_sil  = frame_val_extract(kmeans_pca_cluster_hyperparameter_tuning_df,'Adjusted_rand_score')
dbscan_results_best_sil  = frame_val_extract(dbscan_results,'adjusted_rand_score')

In [ ]:
score_names =['Homogeneity Score','Completeness Score','V_measure Score','Adjusted Rand Score','Silhouette Score','Calinski-Harabasz Score','David-Bouldin Score']
column_names = ['Spectral Clustering','PCA Spectral Clustering','Agglomerative Clustering','PCA Agglomerative Clustering',
                'Gaussian Mixture','PCA Gaussian Mixture','Kmeans','PCA Kmeans','PCA DBSCAN']
#Best Silhouette scores for spectral clustering
spec_arr =[spectral_clustering_sil_best['Homogeneity Score'].iloc[0]
,spectral_clustering_sil_best['Completeness Score'].iloc[0]
,spectral_clustering_sil_best['V_measure Score'].iloc[0]
,spectral_clustering_sil_best['Adjusted_rand_score'].iloc[0]
,spectral_clustering_sil_best['Silhouette Score'].iloc[0]
,spectral_clustering_sil_best['Calinski-Harabasz Score'].iloc[0]
,spectral_clustering_sil_best['David-Bouldin Score'].iloc[0]]

#Best Silhouette scores for pca spectral clustering
pca_spec_arr = [spectral_clustering_pca_sil_best['Homogeneity Score'].iloc[0]
,spectral_clustering_pca_sil_best['Completeness Score'].iloc[0]
,spectral_clustering_pca_sil_best['V_measure Score'].iloc[0]
,spectral_clustering_pca_sil_best['Adjusted_rand_score'].iloc[0]
,spectral_clustering_pca_sil_best['Silhouette Score'].iloc[0]
,spectral_clustering_pca_sil_best['Calinski-Harabasz Score'].iloc[0]
,spectral_clustering_pca_sil_best['David-Bouldin Score'].iloc[0]]

#Best Silhouette scores for Agglomerative clustering
agglomerative_arr = [agglomerative_sil_best['Homogeneity Score'].iloc[0]
,agglomerative_sil_best['Completeness Score'].iloc[0]
,agglomerative_sil_best['V_measure Score'].iloc[0]
,agglomerative_sil_best['Adjusted_rand_score'].iloc[0]
,agglomerative_sil_best['Silhouette Score'].iloc[0]
,agglomerative_sil_best['Calinski-Harabasz Score'].iloc[0]
,agglomerative_sil_best['David-Bouldin Score'].iloc[0]]

#Best Silhouette scores for PCA Agglomerative clustering
pca_agglomerative_arr =[pca_agglomerative_cluster_hyperparameter_tuning_df['Homogeneity Score'].iloc[0]
,pca_agglomerative_cluster_hyperparameter_tuning_df['Completeness Score'].iloc[0]
,pca_agglomerative_cluster_hyperparameter_tuning_df['V_measure Score'].iloc[0]
,pca_agglomerative_cluster_hyperparameter_tuning_df['Adjusted_rand_score'].iloc[0]
,pca_agglomerative_cluster_hyperparameter_tuning_df['Silhouette Score'].iloc[0]
,pca_agglomerative_cluster_hyperparameter_tuning_df['Calinski-Harabasz Score'].iloc[0]
,pca_agglomerative_cluster_hyperparameter_tuning_df['David-Bouldin Score'].iloc[0]]

#Best Silhouette scores for Gaussian Mixture Model
gmm_arr =[gmm_sil_best['Homogeneity Score'].iloc[0]
,gmm_sil_best['Completeness Score'].iloc[0]
,gmm_sil_best['V_measure Score'].iloc[0]
,gmm_sil_best['Adjusted_rand_score'].iloc[0]
,gmm_sil_best['Silhouette Score'].iloc[0]
,gmm_sil_best['Calinski-Harabasz Score'].iloc[0]
,gmm_sil_best['David-Bouldin Score'].iloc[0]]

#Best Silhouette scores for pca Gaussian Mixture Model
pca_gmm_arr =[gmm_pca_best_sil['Homogeneity Score'].iloc[0]
,gmm_pca_best_sil['Completeness Score'].iloc[0]
,gmm_pca_best_sil['V_measure Score'].iloc[0]
,gmm_sil_best['Adjusted_rand_score'].iloc[0]
,gmm_pca_best_sil['Silhouette Score'].iloc[0]
,gmm_pca_best_sil['Calinski-Harabasz Score'].iloc[0]
,gmm_pca_best_sil['David-Bouldin Score'].iloc[0]]

#Best Silhouette scores for Kmeans
kmeans_arr =[kmean_best_sil['Homogeneity Score'].iloc[0]
,kmean_best_sil['Completeness Score'].iloc[0]
,kmean_best_sil['V_measure Score'].iloc[0]
,kmean_best_sil['Adjusted_rand_score'].iloc[0]
,kmean_best_sil['Silhouette Score'].iloc[0]
,kmean_best_sil['Calinski-Harabasz Score'].iloc[0]
,kmean_best_sil['David-Bouldin Score'].iloc[0]]

#Best Silhouette scores for PCA Kmeans
kmeans_pca_arr =[kmeans_pca_best_sil['Homogeneity Score'].iloc[0]
,kmeans_pca_best_sil['Completeness Score'].iloc[0]
,kmeans_pca_best_sil['V_measure Score'].iloc[0]
,kmeans_pca_best_sil['Adjusted_rand_score'].iloc[0]
,kmeans_pca_best_sil['Silhouette Score'].iloc[0]
,kmeans_pca_best_sil['Calinski-Harabasz Score'].iloc[0]
,kmeans_pca_best_sil['David-Bouldin Score'].iloc[0]]


#Best Silhouette scores for PCA DBSCAN
dbscan_pca =[dbscan_results_best_sil['homogeneity_score'].iloc[0]
,dbscan_results_best_sil['completeness_score'].iloc[0]
,dbscan_results_best_sil['v_measure_score'].iloc[0]
,dbscan_results_best_sil['adjusted_rand_score'].iloc[0]
,dbscan_results_best_sil['silhouette_score'].iloc[0]
,dbscan_results_best_sil['calinski-harabasz_score'].iloc[0]
,dbscan_results_best_sil['david-bouldin_score'].iloc[0]]

pd.DataFrame({'Spectral Clustering':spec_arr,'PCA Spectral Clustering':pca_spec_arr,'Agglomerative Clustering':agglomerative_arr
              ,'PCA Agglomerative Clustering':pca_agglomerative_arr,'Gaussian Mixture':gmm_arr,'PCA Gaussian Mixture':pca_gmm_arr,
              'kmeans':kmeans_arr,'PCA Kmeans':kmeans_pca_arr,'PCA DBSCAN':dbscan_pca},index=score_names)
